In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [2]:
load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')

model = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]   

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website('https://edwarddonner.com')
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [8]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}  
    )
    result = completion.choices[0].message.content
    return json.loads(result)


In [9]:
get_links("https://anthropic.com")

{'links': [{'type': 'company page',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'about page', 'url': 'https://www.anthropic.com/about'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [11]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'customer page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'contact sales page', 'url': 'https://www.anthropic.com/contact-sales'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Team plan
Enterprise plan
Education plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console

In [19]:
get_brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
print(get_brochure_user_prompt("HuggingFace", "https://huggingface.co"))


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
You are looking at a company called: HuggingFace
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nvidia/parakeet-tdt-0.6b-v2
Updated
12 days ago
•
91.8k
•
758
ACE-Step/ACE-Step-v1-3.5B
Updated
3 days ago
•
364
Lightricks/LTX-Video
Updated
7 days ago
•
241k
•
1.42k
nari-labs/Di

In [33]:
def create_brochure(company_name,url):
    response=openai.chat.completions.create(
        model=model,
        messages=[
            {"role":"system","content":get_brochure_system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("Anthropic","https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


# Welcome to Anthropic: Where AI Meets Amusement!

---

### Meet Claude: The AI with Style!

Say hello to **Claude**, our star AI model— think of him as that friend who's always up for a chat but also knows when to pause, reflect, and perhaps grab a snack! With Claude, we’re committed to **responsible AI development** that’s safer than your grandmother’s apple pie recipe.

---

### Our Company Culture: Safety First (But Make it Fun)

At Anthropic, we believe in **serious research with a side of silliness**. We’re like mad scientists, except with fewer lab coats and more coffee! Here’s what you can expect from our quirky culture: 

- **Skeptical optimism**: We embrace the wild world of AI while questioning everything... much like how you question the existence of quiet in a coffee shop.
- **Bold steps forward**: Our team of eccentric geniuses takes calculated leaps into the AI abyss, just like you would when deciding whether or not to try a daring new flavor of ice cream!
- **Collaboration**: We work together like pieces of a jigsaw puzzle— fractious, colorful, and occasionally a little confusing!

---

### Our Customers: Who You Gonna Call?

From students diving into AI academia to enterprises that want to jazz up their coding, our **Claude Product Family** has something for everyone, including:

- **Claude 3.7 Sonnet**: The poetry-loving AI! (He really does know how to rhyme).
- **Claude Code**: For those who need coding help that feels like an AI hand-holding session— without the awkwardness.
- **API Enthusiasts**: If you can spell “API,” you’re halfway to building something amazing with Claude!

---

### Careers: Join the Fun Side of AI!

Got skills in **AI research**, **engineering**, or perhaps a hidden talent for karaoke? We want you! At Anthropic, we aren't just looking for teammates; we want characters who can make even AI laugh (okay, maybe that last part is a stretch).

- **Open Roles**: Check our careers page to see where you might fit! Are you ready for a stimulating work environment where existential discussions about AI happen over coffee? 
- **Anthropic Academy**: Where you can learn to craft your very own Claude-like creations! (Disclaimer: Results may vary; Claude is still the star).

---

### In Conclusion: Join the Adventure!

At Anthropic, we’re **building the future of safe AI** – one pun at a time! If you fancy yourself a bit of a tech wizard with a heart, come join us in our quest for transparency and responsible scaling.

Want to chat or learn more? Let’s 🤖 Grab a coffee with Claude!

---

**Disclaimer**: Anthropic is not responsible for any unexpected existential crises caused by engaging with Claude. Side effects may include philosophical debates and a newfound love for reading embarrassing AI documentation.

In [43]:
def stream_brochure(company_name,url):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {'role':'system','content':get_brochure_system_prompt},
            {'role':'user','content':get_brochure_user_prompt(company_name,url)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown","")
        update_display(Markdown(response),display_id=display_handle.display_id)

In [44]:
stream_brochure("Anthropic","https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'learning resources page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}


# Welcome to Anthropic: Where AI Meets Humanity with a Touch of Humor!

**Let’s Meet Claude!**  
No, not your Uncle Claude who spills soda at every family gathering, but Claude 3.7 Sonnet! Our charming AI model here to help you navigate the world of AI while ensuring you don’t click on the suspicious ads claiming to sell “Space Jam NFTs.”

---

### **What’s Cooking at Anthropic?**

At Anthropic, we believe that AI should serve humanity - not annihilate it in a cinematic disaster! We stand at the frontier of AI safety, working hard to ensure technology evolves responsibly. In other words, no rogue AIs looking to conquer the planet on our watch (sorry, Skynet!).

---

### **Our Culture: Safety First, Fun Always!**

- **Work Hard, Laugh Harder**: We focus on building world-class AI while maintaining a laid-back environment where everyone can bring their weirdest jokes to the table (you’ll hear that pun about AI being “humerus” a lot).
  
- **Intelligent Conversations**: Here you can chat with Claude without the awkwardness of small talk. “So, how's the weather?” becomes a high-minded discourse on existential risks instead!

- **Open Doors and Open Minds**: Our team thrives on collaboration and open discussions. Got a wild idea that might prevent a robot uprising? We want to hear it! Just make sure it doesn’t involve a cat takeover.

---

### **Our Customers: Together with Claude!**

We cater to a diverse crowd, from enterprises longing to unleash Claude's coding prowess to educational institutions looking to level up their curricula. With plans as versatile as a Swiss Army knife, we offer:

- **Team Plans**: Because teamwork makes the dream work! Or at least reduces coffee spills.
- **Enterprise Plans**: The big leagues! Where careful planning helps us avoid awkward AI blunders.
- **Education Plans**: Because AI literacy starts young! Your kid could be the next AI-whisperer!

---

### **Careers with Claude: Join the Fun!**

Would you like to build the future of safe AI with us? We have open roles for those who:

- Can code faster than they can make a cup of coffee!
- Want to engage in impactful research without losing their minds (or humor)!
- Enjoy sharing their thoughts on AI, safety, and why cats truly run the internet.

Our vision aligns compassionately with ambition, and we’re always looking for new recruits! If you think you can handle the heat (or at least, the snacks), check our website!

---

### **In Conclusion: We’re Not Just Bots!**

So, whether you’re looking to explore AI, invest in humanity’s future, or join our quirky team, **Anthropic is the place for you!** Together we can make AI safe, ethical, and just a touch less boring. 

_Step right up, say hi to Claude, and join the revolution!_ 

---
**Anthropic: Building Trust Upon Intelligence.**  
*(Sorry, no robots were harmed in the making of this brochure.)*